In [202]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
from PIL import Image
from utils.denoising_utils import *
from models import *
import math

In [203]:
fname_us = "C:\\Users\\Leo\\Desktop\\FinalYearProject\\Dataf\\k3.jpg"
fname_mr = "C:\\Users\\Leo\\Desktop\\FinalYearProject\\Dataf\\k8.jpg"
fname_fusion = "C:\\Users\\Leo\\Desktop\\FinalYearProject\\DD_remote_fusion_remosensoing.jpg"
print(fname_us)
print(fname_mr)
# DIP_remote_fusion_as.jpg

C:\Users\Leo\Desktop\FinalYearProject\Dataf\k3.jpg
C:\Users\Leo\Desktop\FinalYearProject\Dataf\k8.jpg


In [204]:
us = get_gray_image(fname_us,-1)[0]
mr = get_gray_image(fname_mr,-1)[0]
us_pil = us.resize((128,128),Image.ANTIALIAS)
us_np = pil_to_np(us_pil)
mr_pil = mr.resize((128,128),Image.ANTIALIAS)
mr_np = pil_to_np(mr_pil)
fusion_pil = Image.open(fname_fusion)
fusion_np = pil_to_np(fusion_pil)[0]
print(fusion_np.shape)
print("222")
print(us_np)

(128, 128)
222
[[[0.49019608 0.5176471  0.49803922 ... 0.         0.         0.        ]
  [0.5137255  0.52156866 0.5019608  ... 0.         0.         0.        ]
  [0.49019608 0.5294118  0.53333336 ... 0.         0.         0.        ]
  ...
  [0.73333335 0.7529412  0.7607843  ... 0.         0.         0.        ]
  [0.6627451  0.67058825 0.6745098  ... 0.         0.         0.        ]
  [0.10196079 0.09803922 0.10196079 ... 0.         0.         0.        ]]]


In [205]:
A_coeffs = pywt.dwt2(us_np,'haar')
B_coeffs = pywt.dwt2(mr_np,'haar')
F_coeffs = pywt.dwt2(fusion_np,'haar')
A_cAi,(A_cHi,A_cVi,A_cDi) = A_coeffs
B_cAi,(B_cHi,B_cVi,B_cDi) = B_coeffs
F_cAi,(F_cHi,F_cVi,F_cDi) = F_coeffs
print(A_cVi[0][3][4])
print(A_cHi.shape)

-0.009803951
(1, 64, 64)


In [206]:
def find_EP_xy(A_coeffs,B_coeffs,F_coeffs):
    A_cAi,(A_cHi,A_cVi,A_cDi) = A_coeffs
    B_cAi,(B_cHi,B_cVi,B_cDi) = B_coeffs
    F_cAi,(F_cHi,F_cVi,F_cDi) = F_coeffs

    A_cAi,(A_cHi,A_cVi,A_cDi) = A_cAi[0],(A_cHi[0],A_cVi[0],A_cDi[0])
    B_cAi,(B_cHi,B_cVi,B_cDi) = B_cAi[0],(B_cHi[0],B_cVi[0],B_cDi[0])
    # F_cAi,(F_cHi,F_cVi,F_cDi) = F_cAi[0],(F_cHi[0],F_cVi[0],F_cDi[0])

    # A_cAi,(A_cHi,A_cVi,A_cDi) = A_cAi[0]*1000,(A_cHi[0]*1000,A_cVi[0]*1000,A_cDi[0]*1000)
    # B_cAi,(B_cHi,B_cVi,B_cDi) = B_cAi[0]*1000,(B_cHi[0]*1000,B_cVi[0]*1000,B_cDi[0]*1000)
    # F_cAi,(F_cHi,F_cVi,F_cDi) = F_cAi*1000,(F_cHi*1000,F_cVi*1000,F_cDi*1000)
    
    
    w_A = np.square(A_cHi)+np.square(A_cVi)+np.square(A_cDi)
    w_B = np.square(B_cHi)+np.square(B_cVi)+np.square(B_cDi)

    bottom_sum = 0
    top_sum = 0
    for i in range(len(w_A)):
        for j in range(len(w_A[0])):
            Fa_AF_H_xy = math.exp(-abs(A_cVi[i][j]-F_cVi[i][j]))
            Fa_AF_V_xy = math.exp(-abs(A_cHi[i][j]-F_cHi[i][j]))
            Fa_AF_D_xy = math.exp(-abs(A_cDi[i][j]-F_cDi[i][j]))
            EP_AF_xy = (Fa_AF_D_xy+Fa_AF_H_xy+Fa_AF_V_xy)/3

            Fa_BF_H_xy = math.exp(-abs(B_cVi[i][j]-F_cVi[i][j]))
            Fa_BF_V_xy = math.exp(-abs(B_cHi[i][j]-F_cHi[i][j]))
            Fa_BF_D_xy = math.exp(-abs(B_cDi[i][j]-F_cDi[i][j]))
            EP_BF_xy = (Fa_BF_D_xy+Fa_BF_H_xy+Fa_BF_V_xy)/3

            top_sum+=EP_AF_xy*w_A[i][j]+EP_BF_xy*w_B[i][j]

            bottom_sum+=w_A[i][j]+w_B[i][j]

    return top_sum/bottom_sum  
    


In [207]:
iLoop = find_EP_xy(A_coeffs,B_coeffs,F_coeffs)

In [208]:
A_coeffs_ii = pywt.dwt2(A_cAi,'haar')
B_coeffs_ii = pywt.dwt2(B_cAi,'haar')
F_coeffs_ii = pywt.dwt2(F_cAi,'haar')
iiLoop = find_EP_xy(A_coeffs_ii,B_coeffs_ii,F_coeffs_ii)
print(iiLoop)

0.9262630831802092


In [209]:
print((iLoop**(2/3))*(iiLoop**(1/3)))

0.9368921809853482
